In [3]:
import bs4

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import WebBaseLoader

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

from dotenv import load_dotenv

load_dotenv()

True

WebBaseLoader는 WEB의 URL을 주면, 페이지를 가져오는 역할을 함
- 외부에서 가져오는 문서여서, RAG라는 형식으로 사용함

In [4]:
llm = ChatOpenAI(model="gpt-4o")

### <font color=green>llama를 쓰는 경우 아래와 같이 패키지 설치해줘야함</font>

In [5]:
# !pip install bitsandbytes==0.40.0 einops==0.6.1

In [6]:
# !pip install accelerate

In [7]:
# from torch import cuda, bfloat16
# import transformers

# #model_id = 'meta-llama/Llama-2-7b-chat-hf'
# model_id = 'meta-llama/Meta-Llama-3-8B'

# device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# # set quantization configuration to load large model with less GPU memory
# # this requires the `bitsandbytes` library
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

# # begin initializing HF items, you need an access token
# hf_auth = '[HF_AUTH]'
# model_config = transformers.AutoConfig.from_pretrained(
#     model_id,
#     use_auth_token=hf_auth
# )

# model = transformers.AutoModelForCausalLM.from_pretrained(
#     model_id,
#     trust_remote_code=True,
#     config=model_config,
#     quantization_config=bnb_config,
#     device_map='auto',
#     use_auth_token=hf_auth
# )

### <font color=yellow>1. indexing: Load</font>
- DocumentLoaders를 사용하여 블로그 내용을 로드
- Documents는 소스에서 데이터를 로드하고 Documents 목록을 반환하는 개체
- Document는 page_content(str)와 메타데이터(dict)를 포함하는 개체
- urllib을 사용하여, 웹 URL에서 HTML을 로드하고 BeautifulSoup을 사용하여 텍스트로 구문 분석함
    - bs_kwags를 통해 BeautifulSoup 파서에 매개변수를 전달하여 HTML -> 텍스트 구문 분석을 사용자 정희할 수 있음
    - (BeautifulSoup 문서 참조)
    - 이 경우 클래스 "post-content", "post-title" 또는 "post-header"가 있는 HTML 태그만 관련되므로 다른 모든 것을 제거함

In [ ]:
# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [9]:
len(docs[0].page_content)
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


### <font color=yellow>2. indexing: Split</font>
- 로드된 문의 길이는 42,000자를 넘음
- 이는 너무 길어서 많은 모델의 컨텍스트 창에 맞지 않음 
- 컨텍스트 창에 전체 게시물을 맞출 수 있는 모델의 경우에도 모델은 매우 긴 입력에서 정보를 찾는 데 어려움을 겪을 수 있음

- Document를 처리하기 위해 임베딩 및 벡터 저장을 위해 청크로 분할
- 이는 런타임 시 블로그 게시물의 가장 관련성이 높은 부분만 검색하는 데 도움이 됨

- 이 경우 문서를 1000자의 청크로 분할하고 청크 사이에 200자가 겹치도록 하겠음 
- 중복은 진술과 관련된 중요한 맥락에서 진술을 분리할 가능성을 완화하는 데 도움이 됨
- 우리는 각 청크가 적절한 크기가 될 때까지 새 줄과 같은 공통 구분 기호를 사용하여 문서를 재귀적으로 분할하는 RecursiveCharacterTextSplitter를 사용함 
- 이는 일반적인 텍스트 사용 사례에 권장되는 텍스트 분할기임

- add_start_index=True로 Document 내에서 각 분할 Document가 시작되는 문자 인덱스가 메타데이터 속성 "start_index"로 유지되도록 설정함

In [ ]:
# GPT는 4000 tokens 정도됨 (Palcon이 8000 tokens 정도)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

데이터 splitter를 이용해 블록화를 시키면, 잘린 블록마다 메타데이터를 가지게 됨
- 긴 문서중에, 내가 찾은 이 블록이 몇 번째 블록이고, 어디서 부터 시작하고, 이런 소스 데이터를 가지게됨
- 사용자 검색이 들어오면, 유사도 측정을 하고, 그것이 답이다라고 답변을 줄때, 이 메타데이터를 함께 줄 수 있음
- 내가 지금 만든 이 문장이 어느 부분에 어느 포인트를 가지고 이렇게 문장을 만들었어 라고 인지하기 위해 메타데이터를 만듦

In [11]:
len(all_splits)
len(all_splits[0].page_content)
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 8436}

### <font color=yellow>3. indexing: Store</font>
- 이제 런타임에 검색할 수 있도록 667개의 텍스트 청크를 인덱싱해야 함 
- 이를 수행하는 가장 일반적인 방법은 분할된 각 문서의 내용을 포함하고 이러한 포함을 벡터 데이터베이스(또는 벡터 저장소)에 삽입함
- 분할을 검색하려는 경우 텍스트 검색 쿼리를 가져와 이를 포함하고 일정의 "유사성" 검색을 수행하여 쿼리 포함과 가장 유사한 임베딩이 있는 저장된 분할을 식별함 
- 가장 간단한 유사성 측정은 코사인 유사성
    - 즉, 각 임베딩 쌍(고차원 벡터) 사이의 각도의 코사인을 측정함

- Chroma 벡터 저장소와 OpenAIEmbeddings 모델을 사용하여 단일 명령에 모든 문서 분할을 포함하고 저장할 수 있음

In [12]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())         #
#vectorstore = Chroma.from_documents(documents=all_splits, embedding=FastEmbedEmbeddings())     # FastEmbedEmbeddings 오류로 OpenAIEmbeddings로 수정

### <font color=yellow>4. Retrieval and Generation Retrieve</font>
- 이제 실제 애플리케이션 로직을 작성하곘음 
- 우리는 사용자 질문을 받아 해당 질문과 관련된 문서를 검색하고 검색된 문서와 초기 질문을 모델에 전달하고 답변을 반환하는 간단한 애플리케이션을 만들고 싶음

- 먼저 문서 검색을 위한 논리를 정의해야 함 
- LangChain은 문자열 쿼리가 주어지면 관련 문서를 반환할 수 있는 인덱스를 래핑하는 Retriever 인터페이스를 정의햐여험

- 리트리버의 가장 일반적인 유형은 벡터 스토어의 유사성 검색 기능을 사용하여 검색을 용이하게 하는 벡터 스토어 리트리버임 - VectorStore.as_retriever()를 사용하면 벡터 스토어를 쉽게 리트리버로 변환할 수 있음

In [13]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
len(retrieved_docs)

6

In [14]:
print(retrieved_docs[0].page_content)

Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.


### <font color=yellow>5. Retrieval and Generation: Generate</font>
- 질문을 받고, 관련 문서를 검색하고, 프롬프트를 구성하고, 이를 모델에 전달
- 출력을 구문 분석하는 체인에 모두 함께 넣겠음

<font color=green>rlm/rag-prompt를 가져다가 프롬프트 형식으로 사용할 것임</font>

In [15]:
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

c:\Users\jjsd4\workspace\torch\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [16]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [17]:
#vector_store = Chroma.from_documents(documents=chunks, embedding=FastEmbedEmbeddings())
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 3,
        "score_threshold": 0.5,
    },
)

chain = ({"context":retriever, "question": RunnablePassthrough()}
              | prompt
              | llm
              | StrOutputParser())

chain.invoke("What is Task Decomposition?")

'Task Decomposition involves breaking down a complex task into smaller, more manageable steps to enhance model performance. Techniques like Chain of Thought and Tree of Thoughts use this method to interpret the model’s thinking process and explore multiple reasoning possibilities. It can be achieved through simple prompts, task-specific instructions, or human inputs.'

## <font color=yellow>나중에 LLaMA 3.1 8B로 모델만 바꿔서 해보기</font>